# 要約 
このJupyterノートブックは、ファインチューニングされたLlama-3 8bモデルを利用して、大規模なデータセット（25,000サンプル）に対する推論を行うことを目的としています。使用する環境は、T4 GPUを2台使用しており、トレーニング済みのモデルへのアクセスが必要です。

### 取り組む問題
ノートブックは、特に大規模なテストセットに対して各応答モデルの好みを予測し、どのモデルが選ばれるかを判断することに焦点を当てています。具体的には、ユーザーのプロンプトに対してモデルAとモデルBの応答を比較し、その選好を予測します。

### 手法
1. **ライブラリのインポート**:
   - `torch`, `sklearn`, `numpy`, `pandas`などのライブラリを使用。
   - Hugging Faceの`transformers`および`peft`ライブラリを使用して、事前学習済みのLlamaモデルのトークナイゼーションと推論を行います。

2. **データの準備**:
   - テストデータセットから必要なカラムを読み込み、文字列を処理して適切な形式に変換します。

3. **トークナイズ**:
   - `AutoTokenizer`を用いて入力テキストをトークン化し、PyTorchテンソル形式に変換します。

4. **モデルのロード**:
   - 2台のGPUそれぞれに異なるモデルを配置し、`AutoModelForSequenceClassification`を用いてモデルを読み込みます。

5. **LoRa設定**:
   - 減少されたメモリフットプリントでファインチューニングを行うために、LoRa（Low-Rank Adaptation）を設定します。

6. **推論**:
   - データフレームをバッチ処理しながら、モデルに対する推論を実行します。そして、モデルAやB、引き分けの確率を計算します。

7. **マルチスレッド推論**:
   - 結果を並行して計算するために、Pythonのスレッドを使用し、2つのサブセットに分割したデータを各モデルで処理します。

8. **結果の統合**:
   - 最終的にモデルの出力を統合し、提出用のCSVファイルに保存します。

推論は約4.5時間で完了しますが、さらなる改善の余地があるため、異なる後処理方法の検討が提案されています。全体として、このノートブックは大規模言語モデルを用いたユーザー選好予測の実装例を示しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づいて、初心者がつまずきそうな専門用語の解説です。

1. **llama-3**:
   - OpenAIが開発した大規模言語モデルの一種で、特に特定のタスク（この場合は一対の応答の比較）に対する性能を向上させるためにファインチューニングされたモデルです。

2. **ファインチューニング**:
   - 既存のモデルに追加のトレーニングを行って、特定のタスクのパフォーマンスを向上させるプロセス。ベースモデルのパラメータを最適化することによって、特定のデータセットに対する適応性を持たせる。

3. **AutoTokenizer**:
   - Hugging FaceのTransformersライブラリの一部で、自動的にモデルに適したトークナイザを取得するためのクラス。このクラスは、入力テキストをトークン（数値の配列）に変換する機能を提供します。

4. **BitsAndBytesConfig**:
   - 計算量を削減しメモリ使用量を効率化するための設定クラス。特に、大規模なモデルを軽量化するための手法の一つ（例：量子化）を実現するために用いられます。

5. **量子化**:
   - モデルのパラメータを低精度の形式（例えば、8ビットまたは4ビット）で表現するプロセス。これにより、メモリ footprint を削減して推論速度を向上させることができます。

6. **LoRa (Low-Rank Adaptation)**:
   - モデルの効率的なファインチューニング手法の一つです。特に、全結合層の重みを低ランクに近似することで、トレーニングに必要なパラメータの数を減らします。

7. **LoraConfig**:
   - LoRaの設定を定義するためのクラスで、特定のローレベルのパラメータ（例えば、ランクやドロップアウト率）を指定します。

8. **PeftModel**:
   - Parameter Efficient Fine-Tuningの略で、ファインチューニングを行う際に必要なパラメータを効率良く管理するためのモデルクラス。

9. **autocast**:
   - PyTorchの機能で、モデルの推論やトレーニングを実行する際に、自動的に混合精度計算を行うことで性能を向上させます。これにより、速度とメモリ効率が改善されることがあります。

10. **スレッド**:
    - プログラムの実行単位であり、複数のスレッドを用いて同時に処理を行うことで、計算の効率を向上させます。このノートブックでは、異なるモデルを複数のスレッドで並行して扱っています。

11. **attention masks**:
    - トークン化された入力がモデルに与えられる際、どのトークンが実際の入力に該当するのかを示すマスクで、パディングトークンを無視するために使われます。

12. **バッチサイズ (BATCH_SIZE)**:
    - 推論やトレーニングの際に一度に処理するデータのサンプル数のこと。大きなバッチサイズは計算効率を高めますが、メモリ使用量が増えるため、適切なサイズ設定が重要です。

これらの用語に関する理解が深まることで、ノートブック内でのコードの意味がより明確になり、全体の流れを理解する手助けになるでしょう。

---


# 推論 - llama-3 8b 超高速 🚀
このノートブックでは、T4 GPUを2台使ってファインチューニング済みのllama-3 8bモデルを用いた推論を行います。このノートブックを作成した理由は、テストサイズが非常に大きい（25,000サンプル）からです。

前提条件: Llama-3へのアクセスが必要です。もし同意書に記入していない場合は、[こちら](https://www.kaggle.com/models/metaresearch/llama-3)に行き、同意書に記入してLlama-3にアクセスしてください。

トレーニングノートブックは[こちら](https://www.kaggle.com/code/kishanvavdara/lmsys-llama-3-tpu-train/notebook)にあります。

役立つと感じたら、ぜひアップボートしてください！

# ライブラリのインポート


In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("申し訳ありませんが、GPUが必要です！")

In [ ]:
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-llama-3-8b-fine-tuned/checkpoint-700/LMSYS/output_v1/checkpoint-700'
MAX_LENGTH = 2048
BATCH_SIZE = 4
DEVICE = torch.device("cuda")

# データの準備 


In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

# データフレームの各列に対してprocess関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(sample_sub)
display(test.head(5))

In [ ]:
# モデル用のテキストを準備
test['text'] = 'ユーザーのプロンプト: ' + test['prompt'] +  '\n\nモデル A :\n' + test['response_a'] +'\n\n--------\n\nモデル B:\n'  + test['response_b']
print(test['text'][0])

# トークナイズ


In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

# tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# トークンを生成、パディング、最大長さにトランクションし、PyTorchテンソルとして返す
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]

# モデルの読み込み 
一台のGPUに1つのモデルを読み込みます。  


In [ ]:
# BitsAndBytesの設定
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)

# GPU 0にベースモデルをロード
device0 = torch.device('cuda:0')

base_model_0 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
#     quantization_config=quantization_config,
    num_labels=3,
    device_map='cuda:0',
    use_cache=False,
)

# パディングトークンIDを設定
base_model_0.config.pad_token_id = tokenizer.pad_token_id

# GPU 1にベースモデルをロード
device1 = torch.device('cuda:1')
base_model_1 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
#     quantization_config=quantization_config,
    num_labels=3,
    device_map='cuda:1',
    use_cache=False,
)
base_model_1.config.pad_token_id = tokenizer.pad_token_id

これで、各GPUに1つのモデルを正常にロードできました！

# 重みのロード 


In [ ]:
# LoRaの設定

lora_config = LoraConfig(
    r = 16, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    inference_mode=True,
    task_type = 'SEQ_CLS'
)

In [ ]:
# PEFTモデルを取得
# model_0 = get_peft_model(base_model_0, lora_config).to(device0) 
model_0 = PeftModel.from_pretrained(base_model_0, WEIGHTS_PATH)
# model_0 = model_0.merge_and_unload()
# model_0.config.pad_token_id = tokenizer.pad_token_id
# model_0.config.pretraining_tp = 1
model_0.eval()

# model_1 = get_peft_model(base_model_1, lora_config).to(device1) 
model_1 = PeftModel.from_pretrained(base_model_1, WEIGHTS_PATH)
# model_1 = model_1.merge_and_unload()
# model_1.config.pad_token_id = tokenizer.pad_token_id
# model_1.config.pretraining_tp = 1
model_1.eval()

In [ ]:
# 学習可能なパラメータ
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

# 推論



In [ ]:
import gc
gc.collect()

In [ ]:
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)

    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()
    
    # データフレームをバッチに分割して推論を実行
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        generated_class_a.extend(probabilities[:, 0])  # モデル A の確率
        generated_class_b.extend(probabilities[:, 1])  # モデル B の確率
        generated_class_c.extend(probabilities[:, 2])  # 引き分けの確率
    
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  

    return df

In [ ]:
st = time.time()

N_SAMPLES = len(data)

# データを2つのサブセットに分割
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

# スレッドで推論を実行するための関数
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)

# スレッドからの結果を格納する辞書
results = {}

# スレッドを開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

# すべてのスレッドが終了するのを待つ
t0.join()
t1.join()

# 結果を元のデータフレームに統合
data = pd.concat([results[0], results[1]], axis=0)

print(f"処理完了。総時間: {time.time() - st}")

In [ ]:
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]
display(sample_sub)

In [ ]:
sample_sub.to_csv('submission.csv', index=False)

推論は約4.5時間で完了しますが、まだ改善すべき点があります。異なる後処理を試してみて、ぜひ共有してください。Kaggleのやり方です :)